<a href="https://colab.research.google.com/github/Niloy28/NurseActivityRecognition/blob/master/HAR_paper1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Copy dataset files from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Copy data files
data_file_location = "/content/drive/My Drive/nurse care data/non ambiguous processed" #@param {type:"string"}
dest_path = "/content/" #@param {type:"string"}

!cp -rv "$data_file_location" "$dest_path"
!rm -rf /content/sample_data

# Importing dependencies

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from math import sqrt
import math

# Import dataset

In [5]:
#@title Load datasets from folder
#@markdown Outputs a list of all csv files in folder into <i> data_list </i>

folder_path = "/content/non ambiguous processed" #@param {type : "string"}

import os

data_files = os.listdir(folder_path)
data_list = []

for data_file in data_files:
    full_file_name = os.path.join(folder_path, data_file)

    data_list.append(pd.read_csv(full_file_name))
    print(f"{full_file_name} read and appended.")

/content/non ambiguous processed/user04_corrected.csv read and appended.
/content/non ambiguous processed/user18_corrected.csv read and appended.
/content/non ambiguous processed/user07_corrected.csv read and appended.
/content/non ambiguous processed/user38_corrected.csv read and appended.
/content/non ambiguous processed/user51_corrected.csv read and appended.
/content/non ambiguous processed/lab_corrected.csv read and appended.
/content/non ambiguous processed/user08_corrected.csv read and appended.


In [6]:
dataset = pd.concat(data_list, ignore_index = True)

# Data processing

In [7]:
X = dataset.iloc[:, [0, 1, 2]].values
Y = dataset.iloc[:, 3].values
Y = Y - 1

In [8]:
low_pass_filter = np.zeros((X.shape[0], X.shape[1]))
high_pass_filter = np.zeros((X.shape[0], X.shape[1]))

for i in range(len(X)):
    for j in range(X.shape[1]):
        if i == 0:
            low_pass_filter[i][j] = X[i][j]
            high_pass_filter[i][j] = X[i][j]
        else:
            low_pass_filter[i][j] = 0.8*X[i-1][j] + (1-0.8)*X[i][j]
            high_pass_filter[i][j] = X[i][j] - low_pass_filter[i][j]

## Frame preparation

In [9]:
Fs = 20
frame_size = Fs*4 # 80
hop_size =  40

def get_frames(df, frame_size, hop_size,label_data):

    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - frame_size, hop_size):
        value = df[i: i + frame_size, :]
        #y = df[i: i + frame_size, 1]
        #z = df[i: i + frame_size, 2]

        # Retrieve the most often used label in this segment
        label = stats.mode(label_data[i: i + frame_size])[0][0]
        frames.append([value])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

row_frame, row_label = get_frames(X, frame_size, hop_size, Y)
low_pass_frame, low_pass_label = get_frames(low_pass_filter, frame_size, hop_size, Y)

## Feature calculation

In [10]:
mean = np.mean(low_pass_frame, axis = 1)  # mean along (x,y,z) axis
std = np.std(low_pass_frame, axis = 1)  # standard deviation along (x,y,z) axis

In [11]:
rms_x = []  # root mean square along x axis
value = 0
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][0] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_x.append(value)
    value = 0

rms_y = []  # root mean square along y axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][1] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_y.append(value)
    value = 0

rms_z = []  # root mean square z axis
for i in range(low_pass_frame.shape[0]):
    for j in range(low_pass_frame.shape[1]):
        value = value + (low_pass_frame[i][j][2] **2)
        value = value / (low_pass_frame.shape[1])
    value = sqrt(value)
    rms_z.append(value)
    value = 0


In [12]:
lengths = []  # length of the accelerometer vector
length = 0
nx, ny, nz = row_frame.shape[0], row_frame.shape[1], row_frame.shape[2]
acc_vector = np.reshape(row_frame,(nx, ny*nz))

for i in range(acc_vector.shape[0]):
    count = 0
    for j in range(acc_vector.shape[1]):
        length += acc_vector[i][j]**2
        count += 1

        if count == 3:
            length = sqrt(length)
            lengths.append(length)
            length = 0
            count = 0

In [13]:
avc = []  # avc feature

polling_rate = 4 #Hz

for i in range(0, len(lengths), frame_size):
    value = 0
    iterable = lengths[i:i+frame_size]
    for j in range(len(iterable) - 1, 1, -1):
        value += abs(iterable[j] - iterable[j - 1])
    
    value = value / (frame_size / polling_rate)
    avc.append(value)

In [14]:
max_min = []

for i in range(0, len(lengths), frame_size):
    iterable = lengths[i:i+frame_size]

    max_min.append(max(iterable) - min(iterable))

In [15]:
angle = np.zeros((low_pass_frame.shape[0],low_pass_frame.shape[1],low_pass_frame.shape[2]))  # inclenation angle
angles = []
length = 0
for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        length = low_pass_frame[i,j,0]**2 + low_pass_frame[i,j,1]**2 + low_pass_frame[i,j,2]**2
        length = sqrt(length)
        if length == 0:
            print("length is zero in index row= ",i,"column= ",j)
        else:
            angle_x = math.acos(low_pass_frame[i,j,0]/length)
            angle_y = math.acos(low_pass_frame[i,j,1]/length)
            angle_z = math.acos(low_pass_frame[i,j,2]/length)
            angle[i][j][0] = angle_x
            angle[i][j][1] = angle_y
            angle[i][j][2] = angle_z
nx, ny, nz = angle.shape
angle = np.reshape(angle,(nx,ny*nz))

In [16]:
input_vector = np.zeros((row_frame.shape[0], angle.shape[1] + 11))  # input vector for the algorithm
for i in range(input_vector.shape[0]):
    input_vector[i][0] = mean[i][0]
    input_vector[i][1] = mean[i][1]
    input_vector[i][2] = mean[i][2] 
    input_vector[i][3] = std[i][0]
    input_vector[i][4] = std[i][1]
    input_vector[i][5] = std[i][2]
    input_vector[i][6] = rms_x[i]
    input_vector[i][7] = rms_y[i]
    input_vector[i][8] = rms_z[i]
    input_vector[i][9] = max_min[i]
    input_vector[i][10] = avc[i]

for i in range(angle.shape[0]):
    for j in range(angle.shape[1]):
        input_vector[i][j+11] = angle[i][j]


In [17]:
label = row_label[:]

# Feature scaling

In [18]:
sc = StandardScaler()
input_vector = sc.fit_transform(input_vector)

# Train test split

In [19]:
X_train, X_val, y_train, y_val = train_test_split(input_vector, label, test_size = 0.2, random_state = 0, stratify = label)

# Algorithm

## Random forest

In [20]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 200, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [21]:
y_pred = classifier.predict(X_val)

In [22]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred, y_val)

In [23]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred)

In [25]:
pd.DataFrame(cm)

,0,1,2,3,4,5,6,7,8,9,10
0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,1,24,4,13,0,3,0,0,0,0
3,8,0,27,357,106,0,55,0,0,0,13
4,50,1,148,234,701,2,131,4,5,1,63
5,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,3,3,0,10,0,0,0,0
7,0,0,0,0,0,0,0,3,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0


In [24]:
accuracy

0.5554994954591322

## ANN

In [26]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1], )))
model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=324, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=12, activation='softmax'))

model.compile(optimizer=Adam(learning_rate = 0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs = 50, validation_data= (X_val, y_val), verbose=1)

Epoch 1/50
248/248 [==============================] - 1s 5ms/step - loss: 1.5042 - accuracy: 0.4145 - val_loss: 1.3872 - val_accuracy: 0.4541
Epoch 2/50
248/248 [==============================] - 1s 4ms/step - loss: 1.4035 - accuracy: 0.4483 - val_loss: 1.3801 - val_accuracy: 0.4657
Epoch 3/50
248/248 [==============================] - 1s 4ms/step - loss: 1.3769 - accuracy: 0.4552 - val_loss: 1.3519 - val_accuracy: 0.4576
Epoch 4/50
248/248 [==============================] - 1s 4ms/step - loss: 1.3574 - accuracy: 0.4638 - val_loss: 1.3460 - val_accuracy: 0.4717
Epoch 5/50
248/248 [==============================] - 1s 4ms/step - loss: 1.3468 - accuracy: 0.4676 - val_loss: 1.3487 - val_accuracy: 0.4637
Epoch 6/50
248/248 [==============================] - 1s 4ms/step - loss: 1.3264 - accuracy: 0.4767 - val_loss: 1.3453 - val_accuracy: 0.4657
Epoch 7/50
248/248 [==============================] - 1s 4ms/step - loss: 1.3207 - accuracy: 0.4774 - val_loss: 1.3383 - val_accuracy: 0.4697
Epoch 

## CNN

In [27]:
X_train = X_train.reshape(-1, 251, 1, 1)
X_val = X_val.reshape(-1, 251, 1, 1)


In [28]:
model = Sequential()

model.add(Conv2D(16, (2, 2), activation = 'relu', padding = 'same', input_shape = X_train[0].shape))
model.add(Dropout(0.1))
model.add(Conv2D(32, (2, 2), activation='relu', padding= 'same'))
model.add(Dropout(0.2))
#model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
#model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Dense(128, activation = 'relu'))
#model.add(Dropout(0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))

In [29]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs = 50, validation_data= (X_val, y_val), verbose=1)

Epoch 1/50
248/248 [==============================] - 2s 7ms/step - loss: 1.5594 - accuracy: 0.4225 - val_loss: 1.3556 - val_accuracy: 0.4642
Epoch 2/50
248/248 [==============================] - 1s 5ms/step - loss: 1.4063 - accuracy: 0.4519 - val_loss: 1.3432 - val_accuracy: 0.4728
Epoch 3/50
248/248 [==============================] - 1s 5ms/step - loss: 1.3549 - accuracy: 0.4702 - val_loss: 1.3096 - val_accuracy: 0.4798
Epoch 4/50
248/248 [==============================] - 2s 6ms/step - loss: 1.3212 - accuracy: 0.4854 - val_loss: 1.2949 - val_accuracy: 0.4889
Epoch 5/50
248/248 [==============================] - 1s 6ms/step - loss: 1.3100 - accuracy: 0.4891 - val_loss: 1.2828 - val_accuracy: 0.4879
Epoch 6/50
248/248 [==============================] - 1s 6ms/step - loss: 1.2844 - accuracy: 0.4936 - val_loss: 1.2851 - val_accuracy: 0.4995
Epoch 7/50
248/248 [==============================] - 1s 6ms/step - loss: 1.2651 - accuracy: 0.4980 - val_loss: 1.2778 - val_accuracy: 0.4970
Epoch 